# 数据读入模块

In [71]:
import pandas as pd

train_data = pd.read_csv("data\main\df_train_b1.csv", index_col=0)
test_data = pd.read_csv('data\main\df_test_b1.csv', index_col=0)


## 要读入除了main之外的数在此处添加

In [72]:
#df_train = train_data.iloc[:, 0:102]
df_train = train_data
df_test = test_data
# df_train.describe()


In [73]:
#df_train.iloc[:, 3].value_counts()


In [74]:
for i in range(2, 17):
    print(df_train.iloc[:, i].value_counts())


-99     15157
 121       30
 124       29
 115       29
 62        27
        ...  
 267        1
 215        1
 238        1
 320        1
 366        1
Name: basic_2, Length: 344, dtype: int64
-99.0      15156
 0.0        1434
 1000.0       40
 500.0        39
 600.0        34
           ...  
 494.0         1
 1329.0        1
 561.0         1
 678.0         1
 1456.0        1
Name: basic_3, Length: 549, dtype: int64
-99.0      15156
 0.0         921
 600.0        50
 500.0        50
 1000.0       47
           ...  
 405.0         1
 1302.0        1
 434.0         1
 786.0         1
 1456.0        1
Name: basic_4, Length: 718, dtype: int64
 1     10562
 2      5352
 3      1260
 0       428
 4       208
-99      160
 5        25
 6         3
 7         1
Name: basic_5, dtype: int64
 1     8738
 2     6380
 3     2157
 4      473
-99     160
 5       77
 6       12
 7        2
Name: basic_6, dtype: int64
38    857
39    808
34    796
37    767
33    765
42    755
41    736
36    722


In [75]:
full_col_name = list(df_train.columns)
num_col = ["basic_2",
           "basic_3",
           "basic_4",
           "basic_7",
           "basic_13"]
type_col = ["basic_1",
            "basic_10",
            "basic_12",
            "basic_14"]
drop_col = ["basic_8",
            "basic_9",
            "basic_11",
            "basic_15"]
loan_col = [i for i in full_col_name if "loan" in i]
overdue_col = [i for i in full_col_name if "overdue" in i]
query_col = [i for i in full_col_name if "query" in i]

x_col = num_col+type_col+loan_col+overdue_col+query_col
# x_col为模型读入的自变量列名称


In [76]:
# df_train.drop(columns=drop_col)


# 训练模型

## 划分测试集训练集

In [77]:
from sklearn import model_selection


X = df_train[x_col]
Y = df_train["label"]
seed = 666
test_size = 0.1
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=test_size, random_state=seed)


## 训练模型

## 超参数

In [78]:
N = 100
num_round = 300

max_depth = [3, 4, 5]
eta = [0.1]
colsample_bytree = [0.75, 0.7, 0.65]
scale_pos_weight = [1, 2, 3, 4, 5, 6, 7]


In [79]:
import xgboost as xgb
import random


x_perd = xgb.DMatrix(df_test[x_col])
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtrain, 'train'), (dtest, 'eval')]

pred_labels = []
loss_seq = []

for i in range(N):
    param = {
        'max_depth': random.choice(max_depth),
        'eta': random.choice(eta),
        'colsample_bytree': random.choice(colsample_bytree),
        'objective': 'binary:logistic',
        # 'verbosity':0,
        'scale_pos_weight': random.choice(scale_pos_weight),
        'eval_metric': 'auc',
        'seed': 666,
    }

    bst = xgb.train(param, dtrain, num_round, evallist,
                    early_stopping_rounds=5, verbose_eval=False)
    loss = float(bst.eval(dtest).split(":")[1])
    pred_label = list(bst.predict(x_perd))

    loss_seq.append(loss)
    print(i+1, ":", loss, sep="")
    print(param)
    print("-------------------------")
    pred_labels.append([i for i in pred_label])

print('最大AUC:', max(loss_seq), sep="")


C:\Users\tassadar667\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:568: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


1:0.6637588626155388
{'max_depth': 5, 'eta': 0.1, 'colsample_bytree': 0.65, 'objective': 'binary:logistic', 'scale_pos_weight': 7, 'eval_metric': 'auc', 'seed': 666}
-------------------------
2:0.655852110323323
{'max_depth': 3, 'eta': 0.1, 'colsample_bytree': 0.75, 'objective': 'binary:logistic', 'scale_pos_weight': 4, 'eval_metric': 'auc', 'seed': 666}
-------------------------
3:0.6754148260885705
{'max_depth': 3, 'eta': 0.1, 'colsample_bytree': 0.65, 'objective': 'binary:logistic', 'scale_pos_weight': 6, 'eval_metric': 'auc', 'seed': 666}
-------------------------
4:0.6797533316010246
{'max_depth': 3, 'eta': 0.1, 'colsample_bytree': 0.7, 'objective': 'binary:logistic', 'scale_pos_weight': 7, 'eval_metric': 'auc', 'seed': 666}
-------------------------
5:0.6439641412079142
{'max_depth': 3, 'eta': 0.1, 'colsample_bytree': 0.75, 'objective': 'binary:logistic', 'scale_pos_weight': 5, 'eval_metric': 'auc', 'seed': 666}
-------------------------
6:0.6935344667582316
{'max_depth': 4, 'eta

In [80]:
# xgb.plot_importance(bst,max_num_features=20)

# 输出模块

In [82]:
import time
import numpy as np
#from sklearn.metrics import roc_auc_score

date = time.strftime("%Y%m%d", time.localtime())

# 选择前n个结果进行平均
n = 3
res_num = np.argsort([-i for i in loss_seq])[0:min(n, N)]

final_pred_label = np.mean([pred_labels[i] for i in res_num], axis=0)
#local_auc = roc_auc_score(y_test, final_pred_label)
print("本地测试集前n个auc为", [loss_seq[i] for i in res_num])

res = pd.DataFrame({
    'cust_id': list(df_test.index),
    'label': final_pred_label,
})
res.to_csv("result"+date+".csv", encoding="utf-8", index=False)


本地测试集前n个auc为 [0.7124336463862802, 0.7124336463862802, 0.7124336463862802]
